In [1]:
import torch


In [2]:
# Find what device to work with
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
torch.std_mean(y_train)

NameError: name 'y_train' is not defined

In [4]:
inv_transform.shape

NameError: name 'inv_transform' is not defined

In [8]:
from sklearn.preprocessing import StandardScaler


def normalize(X_train, y_train, X_test, y_test,device):

    scaler_X = StandardScaler()
    scaler_y = StandardScaler()

    X_train = torch.as_tensor(scaler_X.fit_transform(X_train)).float().to(device)
    y_train = torch.as_tensor(scaler_y.fit_transform(y_train)).float().to(device)

    inverse_scaler_y = lambda t: torch.as_tensor(scaler_y.inverse_transform(t)).to(device)

    X_test = torch.as_tensor(scaler_X.transform(X_test)).float().to(device)
    y_test=y_test.float().to(device)
    return X_train, y_train, X_test,y_test , inverse_scaler_y

def get_data(splitting_index,device):
    X_train = torch.load('data/boston_X_train_('+str(splitting_index)+').pt')
    y_train = torch.load('data/boston_y_train_('+str(splitting_index)+').pt')
    X_test = torch.load('data/boston_X_test_('+str(splitting_index)+').pt')
    y_test=torch.load('data/boston_y_test_('+str(splitting_index)+').pt')
    return normalize(X_train, y_train, X_test, y_test,device)

In [9]:
X_train, y_train, X_test, y_test, inv_transform_y= get_data(1,device)

In [12]:
torch.std_mean(inv_transform_y(y_train))

(tensor(9.4673), tensor(22.6422))

In [13]:
torch.std_mean(y_test)

(tensor(6.2773), tensor(21.5569))